## Get ASASSN lightcurves from RA and DEC (and radius in arcmin, default 1 arcmin)
by scraping the https://asas-sn.osu.edu/variables website

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pprint

In [2]:
def download_file(url, file_name):
    # Send GET request
    response = requests.get(url)
    # Save the PDF
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)

In [3]:
def download_closest_ASASSN_lightcurve_with_radec(ra, dec, radius=1, save_lightcurve_csv=True):
    # Request ASASSN lightcurve
    URL = "https://asas-sn.osu.edu/variables?ra="+str(ra)+"&dec="+str(dec)+"&radius="+str(radius)+"&vmag_min=&vmag_max=&amplitude_min=&amplitude_max=&period_min=&period_max=&lksl_min=&lksl_max=&class_prob_min=&class_prob_max=&parallax_over_err_min=&parallax_over_err_max=&name=&references[]=I&references[]=II&references[]=III&references[]=IV&references[]=V&references[]=IX&sort_by=raj2000&sort_order=asc&show_non_periodic=true&show_without_class=true&asassn_discov_only=false&"
    variable_stars_database_page = requests.get(URL)
    soup = BeautifulSoup(variable_stars_database_page.content, "html.parser")
    
    # ASASSN results
    results_table = soup.find_all("table", class_="table-striped")[0]
    table_body = results_table.find_all("tbody")[0]
    table_head = results_table.find_all("thead")[0]
    
    # Parse the results table and download the lightcurve of the closest source
    if(len(table_body.find_all("tr"))>0):
        th = table_head.find_all("th")
        td = table_body.find_all("td")
        first_row = {
            th[0].text: td[0].find_all("a")[0].text,
            'href': "https://asas-sn.osu.edu"+td[0].find_all("a")[0].get("href"),
            th[1].text: td[1].text,
            th[2].text: td[2].text,
            th[3].text: td[3].text,
            th[4].text: td[4].text,
            th[5].text: td[5].text,
            th[6].text: td[6].text,
            th[7].text: td[7].text,
            th[8].text: td[8].text
        }
        # Print the details of the closest source
        pp = pprint.PrettyPrinter(indent=0)
        pp.pprint(first_row)

        # Get the Gaia EDR3 ID of the source
        ## This section can be skipped and the lightcurve can be saved with the ASASSN name
        lightcurve_URL = first_row['href']
        lightcurve_page = requests.get(lightcurve_URL)
        lightcurve_soup = BeautifulSoup(lightcurve_page.content, "html.parser")
        gaia_edr3_id = lightcurve_soup(text = re.compile('EDR3_ID'))[0].parent.parent.parent.find_all("span", class_="star-data__value")[0].text
        gaia_edr3_id = gaia_edr3_id.replace('\n','')
        print("Gaia EDR3 ID: ", gaia_edr3_id)
        
        # Download the lightcurve of the closest source
        if save_lightcurve_csv:
            download_file(first_row['href']+".csv", "EDR3_"+gaia_edr3_id+".csv")
    else:
        print("No Variables found\nRequest: ", URL)

In [4]:
ra = 0.0317     # deg
dec = 34.67396  # deg
radius = 5      # arcmin
download_closest_ASASSN_lightcurve_with_radec(ra, dec, radius)

{'ASAS-SN Name': 'ASASSN-V J000007.61+344026.3',
'Amplitude': '1.25',
'Declination': '34.67396',
'Distance from Search (arcsec)': '0.00',
'Mean VMag': '15.94',
'Other Names': 'SERIV 27',
'Period': '0.4619591',
'Right Ascension': '0.0317',
'Type': 'RRAB',
'href': 'https://asas-sn.osu.edu/variables/8b0b017b-533b-57b4-9afb-3c466701cdaa'}
Gaia EDR3 ID:  2875434876156537472
